In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('/content/covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop('has_covid', axis=1), df['has_covid'], test_size=0.2)

In [7]:
x_train

,age,gender,fever,cough,city
50,19,Male,101.0,Mild,Delhi
57,49,Female,99.0,Strong,Bangalore
71,75,Female,104.0,Strong,Delhi
41,82,Male,NaN,Mild,Kolkata
32,34,Female,101.0,Strong,Delhi
...,...,...,...,...,...
72,83,Female,101.0,Mild,Kolkata
60,24,Female,102.0,Strong,Bangalore
42,27,Male,100.0,Mild,Delhi
5,84,Female,NaN,Mild,Bangalore


Without Column Transformer

In [9]:
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.transform(x_test[['fever']])

In [10]:
x_train['fever']

50    101.0
57     99.0
71    104.0
41      NaN
32    101.0
      ...  
72    101.0
60    102.0
42    100.0
5       NaN
92    102.0
Name: fever, Length: 80, dtype: float64

In [11]:
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.transform(x_test[['cough']])

In [12]:
ohe = OneHotEncoder(drop='first', sparse=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])
x_test_gender_city = ohe.transform(x_test[['gender', 'city']])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
x_train_age = x_train.drop(columns=['gender', 'fever', 'cough', 'city']).values
x_test_age = x_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

In [14]:
x_train_transformed = np.concatenate((x_train_age, x_train_fever, x_train_cough, x_train_gender_city), axis=1)
x_test_transformed = np.concatenate((x_test_age, x_test_fever, x_test_cough, x_test_gender_city), axis=1)

In [17]:
x_train_transformed

array([[ 19.        , 101.        ,   0.        ,   1.        ,
          1.        ,   0.        ,   0.        ],
       [ 49.        ,  99.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 75.        , 104.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 82.        , 101.04225352,   0.        ,   1.        ,
          0.        ,   1.        ,   0.        ],
       [ 34.        , 101.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 20.        , 101.04225352,   1.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [ 74.        , 102.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [ 49.        , 101.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 38.        , 101.04225352,   0.        ,   1.        ,
          1.    

In [18]:
x_train_transformed_df = pd.DataFrame(x_train_transformed)

In [19]:
x_train_transformed_df.head()

,0,1,2,3,4,5,6
0,19.0,101.000000,0.0,1.0,1.0,0.0,0.0
1,49.0,99.000000,1.0,0.0,0.0,0.0,0.0
2,75.0,104.000000,1.0,0.0,1.0,0.0,0.0
3,82.0,101.042254,0.0,1.0,0.0,1.0,0.0
4,34.0,101.000000,1.0,0.0,1.0,0.0,0.0


With Column Transformer

In [20]:
from sklearn.compose import ColumnTransformer

In [22]:
transformer = ColumnTransformer(transformers = [
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')

In [23]:
transformer.fit_transform(x_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  19.        ],
       [ 99.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  49.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  75.        ],
       [101.04225352,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [101.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [101.04225352,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  20.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  74.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [101.04225352,   0.        ,   1.        ,   1.        ,
          0.    

In [24]:
transformer.transform(x_test)

array([[104.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  16.        ],
       [101.04225352,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [102.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  69.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  18.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  31.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [ 98.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  40.        ],
       [ 99.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  59.        ],
       [ 99.        ,   0.        ,   1.        ,   1.        ,
          0.    

In [26]:
column_transformer_df = pd.DataFrame(transformer.fit_transform(x_train))
column_transformer_df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6
0,101.000000,0.0,1.0,1.0,0.0,0.0,19.0
1,99.000000,1.0,0.0,0.0,0.0,0.0,49.0
2,104.000000,1.0,0.0,1.0,0.0,0.0,75.0
3,101.042254,0.0,1.0,0.0,1.0,0.0,82.0
4,101.000000,1.0,0.0,1.0,0.0,0.0,34.0


In [28]:
column_transformer_df.shape

(80, 7)